## initial mapping workflow for supratidal forests (STF)
- works on geojson GA coastal tiles `./geojson/ga_summary_grid_c3_coastal.geojson`
- select `time_range` and if to `export` results as tif

In [1]:
%matplotlib inline

import sys
from osgeo import gdal
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
# pd.set_option('display.max_rows', None)

sys.path.insert(1, "/home/jovyan/code/dea-notebooks/Tools")
import datacube
from dea_tools.plotting import display_map
from datacube.utils.cog import write_cog
from datacube.utils.geometry import Geometry
from dea_tools.spatial import xr_rasterize
dc = datacube.Datacube()

# load virtual product
import importlib
from datacube.virtual import catalog_from_file
from datacube.virtual import DEFAULT_RESOLVER
catalog = catalog_from_file('../virtual_products/virtual_product_cat_wcf.yaml')

## user inputs: tile, time and export geotiffs?

In [2]:
# add time (not a range, just repeat year input here)
time_range = ("2020", "2020")

# export as geotiff?
export = True

### using GA coastal tiles geojson

In [3]:
# GA coastal tiles
vector_file = './geojson/ga_summary_grid_c3_coastal.geojson'
attribute_col = 'geometry'

gdf = gpd.read_file(vector_file)
mainland_grid = gdf[gdf['type'] == 'mainland']

id_list = []
for index, row in mainland_grid.iterrows():
    id_list.append(row['id'])
mainland_grid.explore()

### select the tile of interest (click on map above and find value for `id` and manually select using `loc` below)

In [4]:
gdf = mainland_grid.loc[mainland_grid['id'] == 26]

In [5]:
gdf.explore()

In [6]:
# Create the 'query' dictionary object
res = (-30, 30)

query = {
    "time": time_range,
    'resolution':res}

In [7]:
# Loop through polygons in geodataframe and add geom to queries
for index, row in gdf.iterrows():
    print(f'Feature: {index + 1}/{len(gdf)}')
    
    # Extract the feature's geometry as a datacube geometry object
    geom = Geometry(geom=row.geometry, crs=gdf.crs)
    
    # Update the query to include our geopolygon
    query.update({'geopolygon': geom})

# Extracting specific keys from dictionary (removing time to load things like item and srtm)
query_notime = {key: query[key] for key in query.keys()
       & {'resolution', 'geopolygon'}}

Feature: 26/1


### load strm, item, mangroves

In [8]:
# Load STRM
srtm_ds = dc.load(product = 'ga_srtm_dem1sv1_0', output_crs="EPSG:3577", **query_notime)
srtm = srtm_ds.dem_h

# Load item
item_ds = dc.load(product = 'item_v2', output_crs="EPSG:3577", **query_notime)
item = item_ds.relative

# Load in mangrove cover
DEAmangrove = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', output_crs="EPSG:3577", **query)

# if no mangroves within AOI, create dummy xr.dataarray
if DEAmangrove.data_vars == {}:
    mangrove = xr.DataArray(np.zeros_like(srtm), coords=srtm.coords, dims=srtm.dims, attrs=srtm.attrs)
else:
    # get output of mangrove == 1, not mangrove == 0
    mangrove = (DEAmangrove.canopy_cover_class != 255)

### load in woody cover virtual product

In [9]:
# load in vp woody cover
sys.path.insert(1, '../virtual_products') 
# The dictionary which datacube uses to understand (resolve) the different virtual product functionality
# Need to add any aggregations for the VP you're using

# Get location of transformation
aggregation = "best_pixel_gmad"
agg_loc = importlib.import_module(aggregation)
agg_class = aggregation.split('.')[-1]

DEFAULT_RESOLVER.register('aggregate', agg_class, getattr(agg_loc, agg_class) )

In [10]:
# Need to add any tranformations for the VP you're using

# Get location of transformation
transformation = "woody_cover"
trans_loc = importlib.import_module(transformation)
trans_class = transformation.split('.')[-1]

DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

In [11]:
# load woody_cover vp
product = catalog['woody_cover']
woody_cover_vp = product.load(dc, **query)

# # woody cover with threshold for saltmarsh
woody_cover = xr.where(woody_cover_vp.woody_cover > 0.5, 1, 0).astype('int8')
woody_cover.attrs['crs'] = 'EPSG:3577'

/env/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


### create initial supratidal forest extent product

In [12]:
## wrangle binary maps and create STF mask

# not mangrove == True
not_mangrove = (1 - mangrove)
not_mangrove = not_mangrove == 1

# greater than 1m AHD and less than 10m AHD == True
AHD_min = 1
AHD_max = 10

# SRTM
lessthan10m_AHD = srtm <= AHD_max
greaterthan1m_AHD = srtm >= AHD_min
AHD_threshold_srtm = lessthan10m_AHD & greaterthan1m_AHD

# item (remove all intertidal areas)
not_intertidal = item == 9

# remove time dim
not_mangrove = not_mangrove.squeeze('time')
AHD_threshold_srtm = AHD_threshold_srtm.squeeze('time')
not_intertidal = not_intertidal.squeeze('time')

In [13]:
# combine masks
STF = xr.where((not_mangrove == False) + (AHD_threshold_srtm == False) + (not_intertidal == False) + (woody_cover == False), 0, 1).astype('int8')

In [14]:
# Generate a polygon mask to keep only data within the polygon
mask = xr_rasterize(gdf, srtm_ds)

# Mask dataset to set pixels outside the polygon to `NaN`
STF = STF.where(mask)

### combine inputs and output to 7 band xarray dataset
- band 1: mangrove (1), no mangrove (0)
- band 2: SRTM data (raw elevation values)
- band 3: SRTM threshold between 1-10 (1), outside this range (0)
- band 4: ITEMs intertidal areas (1), non intertidal areas (0)
- band 5: woody cover fraction data (raw fraction values 0-1)
- band 6: woody cover fraction threshold >0.5 (1), <0.5 (0)
- band 7: STF present (1), STF absent (0)

In [15]:
# make all into datasets and merge
variables_xarray_list = []

# band 1
mangrove_ds = mangrove.to_dataset(name="mangrove").squeeze().drop('time')
variables_xarray_list.append(mangrove_ds)

# band 2
srtm_false_value_remove = xr.where(srtm < -10 ,0, srtm.values)
srtm_ds = srtm_false_value_remove.to_dataset(name="SRTM").squeeze().drop('time')
variables_xarray_list.append(srtm_ds)

# band 3
AHD_threshold_srtm_ds = AHD_threshold_srtm.to_dataset(name="SRTM_threshold").squeeze().drop('time')
variables_xarray_list.append(AHD_threshold_srtm_ds)

# band 4
intertidal = xr.where(not_intertidal == True, 0, 1).astype('int8')
intertidal_ds = intertidal.to_dataset(name="intertidal_and_below").squeeze().drop('time')
variables_xarray_list.append(intertidal_ds)

# band 5
woody_cover_raw = woody_cover_vp.woody_cover
woody_cover_raw_mask_intertidal = woody_cover_raw.where(not_intertidal) # mask by intertidal due to WCF values in water that are erronous (note this means areas in the intertidal are not masked for WCF, which we actually may want to know about for STF vs saltmarsh)
woody_cover_raw_mask_intertidal_ds = woody_cover_raw_mask_intertidal.to_dataset(name="woody_cover").squeeze()
variables_xarray_list.append(woody_cover_raw_mask_intertidal_ds)

# band 6
woody_cover_threshold_ds = woody_cover.to_dataset(name="woody_cover_threshold").squeeze()
variables_xarray_list.append(woody_cover_threshold_ds)

# band 7
STF_ds = STF.to_dataset(name="STF").squeeze().drop('time')
variables_xarray_list.append(STF_ds)

stacked_xarray = xr.merge(variables_xarray_list)

### output to geotiff

In [16]:
def set_band_names(input_img: str, band_names: list, feedback: bool = False):
    """
    NOTE: Function is from RSGISlib that is not avaliable on DEA sandbox
    A utility function to set band names.

    :param input_img: input image file.
    :param band_names: is a list of band names
    :param feedback: is a boolean specifying whether feedback will be printed to the
                     console (True= Printed / False (default) Not Printed)

    .. code:: python

        from rsgislib import imageutils

        input_img = 'injune_p142_casi_sub_utm.kea'
        band_names = ['446nm','530nm','549nm','569nm','598nm','633nm','680nm','696nm',
                      '714nm','732nm','741nm','752nm','800nm','838nm']

        imageutils.set_band_names(input_img, band_names)

    """
    dataset = gdal.Open(input_img, gdal.GA_Update)

    for i in range(len(band_names)):
        band = i + 1
        bandName = band_names[i]

        imgBand = dataset.GetRasterBand(band)
        # Check the image band is available
        if not imgBand is None:
            if feedback:
                print('Setting Band {0} to "{1}"'.format(band, bandName))
            imgBand.SetDescription(bandName)
        else:
            raise rsgislib.RSGISPyException("Could not open the image band: ", band)


In [17]:
if export == False:
    pass
else:
    write_cog(geo_im=stacked_xarray.to_array(),
              fname='initial_STF_model_coastal_tile_gridID_' + gdf['id'].astype(str).item() +'_year_' + time_range[0] + '.tif',
              overwrite=True,
              nodata=0.0)

    input_img = 'initial_STF_model_coastal_tile_gridID_' + gdf['id'].astype(str).item() +'_year_' + time_range[0] + '.tif'
    band_names = list(stacked_xarray.data_vars.keys())
    set_band_names(input_img, band_names)